In [1]:
%load_ext autoreload
%autoreload 2

import torch
import yaml
import sys
from omegaconf import OmegaConf
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from tqdm import tqdm
from pathlib import Path
import os.path as osp
import fsspec
import json
sys.path.append('fbsource/fbcode/surreal/')
from maploc.utils.viz_2d import plot_images, plot_keypoints, features_to_RGB, save_plot, add_text
from maploc.osm.viz import Colormap, plot_nodes
from maploc.data.torch import collate
from maploc.utils.geo import BoundaryBox, Projection
from maploc.utils.io import write_json
from maploc.osm.tiling_v2 import TileManager

colormap = Colormap()
torch.set_grad_enabled(False);
plt.rcParams.update({'figure.max_open_warning': 0})

from maploc.data.aria import compute_geoalignment, datasets, FILENAME_GEOALIGN_GPS, FILENAME_GEOALIGN_MAPLOC

# Initial alignment with GPS only

The alignment is automatically written to `output_dir/$FILENAME_GEOALIGN_GPS`. This part does not require a GPU.

In [4]:
scene = "reloc_seattle_downtown"
scene = "reloc_seattle_pike"
scene = "reloc_seattle_westlake"
scene = "reloc_detroit_greektown"
scene = "reloc_detroit_gcp"
scene = "reloc_detroit_cp"

output_dir = Path("/home/psarlin/local/data/aria_dumps_v2/", scene)
tmp_dir = Path("/home/psarlin/local/aria")
tag = datasets[scene]["tag"]
print(scene, tag)

output_dir.mkdir(parents=True, exist_ok=True)
Rt_slam2geo, projection, all_lla_gps, all_xy_gps, all_xy_gt_align, all_indices  = compute_geoalignment(output_dir/FILENAME_GEOALIGN_GPS, tmp_dir, tag=tag, visualize=True)

acc = all_lla_gps[:, -1]
plt.figure(dpi=200)
plt.scatter(*all_xy_gps.T, c=all_indices, cmap="rainbow", s=1, linewidth=0)
plt.gca().set_aspect("equal")
plt.title("Raw GPS measurements")
plt.figure(dpi=200)
plt.scatter(*all_xy_gps.T, c=np.clip(acc / np.percentile(acc, 95), 0, 1), cmap="jet", s=1)
plt.gca().set_aspect("equal")
plt.title("Raw GPS measurements with accuracy")
plt.figure(dpi=200)
plt.scatter(*all_xy_gps[acc > 15].T, color="red", linewidth=0, s=1, label="accuracy>15m")
plt.scatter(*all_xy_gps[acc < 15].T, color="blue", linewidth=0, s=1, label="accuracy<15m")
plt.legend()
plt.gca().set_aspect("equal")
plt.title("Raw GPS measurements")

bbox_total = BoundaryBox(*np.percentile(all_xy_gt_align, [0, 100], 0))
tiler = TileManager.from_bbox(output_dir, projection, bbox_total + 256, 128, 2)
tile_total = tiler.query(bbox_total + 64)
map_total = Colormap().apply(tile_total.raster)

plot_images([map_total], dpi=200)
plt.plot(*tile_total.to_uv(all_xy_gt_align).T, c='k', lw=0.5);
plt.scatter(*tile_total.to_uv(all_xy_gps).T, c='r', s=1, linewidth=0);

In [ ]:
plot_images([map_total], dpi=200)
plt.plot(*tile_total.to_uv(all_xy_gt_align).T, c='k', lw=0.5);
# plt.scatter(*tile_total.to_uv(all_xy_gps).T, c='k', s=2);
c = tile_total.to_uv(all_xy_gt_align[1200])  # change this index
p = 150
plt.ylim([c[1]+p, c[1]-p])
plt.xlim([c[0]-p, c[0]+p]);

# Refine with predictions

This requires a GPU.

In [ ]:
from maploc.module import GenericModule
cfg = {'model': {"num_rotations": 256, "apply_map_prior": False}}
exper = "bev1-osm2-mly13-n100_res101-vgg16_depth-bins33-nonorm_bs9-resize512_d8-nrot64-normvalid-prior-rep"
model = GenericModule.load_for_evaluation(exper, cfg)

In [ ]:
scene = "reloc_seattle_downtown"
# scene = "reloc_seattle_pike"
scene = "reloc_detroit_greektown"
# scene = "reloc_detroit_gcp"
scene = "reloc_seattle_westlake"

from maploc.data.mapillary import MapillaryDataModule
from maploc.models.sequential import RigidAligner, GPSAligner

# create the dataloader
conf = OmegaConf.load('fbsource/fbcode/surreal/maploc/conf/data/mapillary_v4.yaml')
conf = OmegaConf.merge(conf, OmegaConf.create(yaml.full_load("""
local_dir: "./data/aria_dumps_v2/"
dump_dir: manifold://psarlin/tree/maploc/data/aria_v2
scenes: []
tiles_filename: tiles.pkl
max_init_error: 0
init_from_gps: false
return_gps: true
loading:
    val: {batch_size: 1, num_workers: 0}
    train: ${.val}
random: false
augmentation: {rot90: false, flip: false}
filter_for: null
split: null
resize_image: 512
""")))
conf["scenes"] = [scene]
OmegaConf.resolve(conf)
datamodule = MapillaryDataModule(conf)
datamodule.prepare_data()
datamodule.setup()
dataset, chunk2idx = datamodule.sequence_dataset("val", dict(split_cam=False, max_length=1e9, min_length=1, max_inter_dist=20))
chunks = list(chunk2idx)

# alignment objects
aligner = RigidAligner(track_priors=False, num_rotations=512)
aligner_gps = GPSAligner(track_priors=False, num_rotations=512)

# run over all images
all_xy_gt = []
all_yaw_gt = []
for i in tqdm(range(len(dataset))):
    data = dataset[i]
    data = model.transfer_batch_to_device(data, model.device, i)
    pred = model(collate([data]))

    # mask = torch.zeros(data["map"].shape[-2:], dtype=torch.bool)
    # mask[5:-5, 5:-5] = True
    # pred["log_probs"].masked_fill_(~mask[None, :, :, None], -np.inf)
    canvas = data["canvas"]
    xy, yaw = canvas.to_xy(data["xy"].double()), data["roll_pitch_yaw"][-1].double()
    aligner.update(pred["log_probs"][0], canvas, xy, yaw)
    aligner_gps.update(canvas.to_xy(data["xy_gps"].double()), data["accuracy_gps"], canvas, xy, yaw)
    all_xy_gt.append(xy)
    all_yaw_gt.append(yaw)
aligner.compute()
aligner_gps.compute()

# transform the trajectories
all_xy_gt = torch.stack(all_xy_gt)
all_yaw_gt = torch.stack(all_yaw_gt)
all_xy_seq, all_yaw_seq = aligner.transform(all_xy_gt, all_yaw_gt)
all_xy_gps_align, all_yaw_gps_align = aligner_gps.transform(all_xy_gt, all_yaw_gt)

# visualizations
plot_images([aligner.belief.max(-1).values.cpu(), aligner_gps.belief.max(-1).values.cpu()], cmaps="jet")
print(f"diff: {torch.norm(aligner.Rt_slam2geo[1]).item():.2f}m, {np.rad2deg(torch.atan2(*aligner.Rt_slam2geo[0][[1, 0], [0, 0]]).item()):.2f}deg")

# histogram of pose changes
plt.figure()
plt.hist(np.linalg.norm(all_xy_gt.cpu()-all_xy_seq.cpu(), axis=-1), bins=20);

bbox_total = BoundaryBox(*np.percentile(all_xy_gt.cpu(), [0, 100], 0))
tile_total = dataset.tile_managers[dataset.cfg.scenes[0]].query(bbox_total + 20)
map_total = Colormap().apply(tile_total.raster)
plot_images([map_total], dpi=200)
plt.scatter(*tile_total.to_uv(all_xy_gt.cpu()).T, c="g", linewidth=0, s=1);
plt.scatter(*tile_total.to_uv(all_xy_seq.cpu()).T, c="r", linewidth=0, s=1);
plt.scatter(*tile_total.to_uv(all_xy_gps_align.cpu()).T, c="b", linewidth=0, s=1);

plot_images([map_total], dpi=200)
for idx in chunk2idx.values():
    plt.plot(*tile_total.to_uv(all_xy_seq[idx].cpu()).T, c="r", linewidth=0.5, label="seq fusion");
for idx in chunk2idx.values():
    plt.plot(*tile_total.to_uv(all_xy_gt[idx].cpu()).T, c="k", linewidth=0.5, label="GPS fit");
plt.gca().autoscale(enable=False)
xy_gps = dataset.tile_managers[scene].projection.project(dataset.data["gps_position"][dataset.data["gps_accuracy"]<100].numpy())
plt.scatter(*tile_total.to_uv(xy_gps).T, c="k", linewidth=0, s=2, label="GPS raw")
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys());
plt.title(dataset.cfg.scenes[0]);

# export and write to manifold to perform a second processing of the data
with fsspec.open(osp.join(dataset.cfg.dump_dir, scene, FILENAME_GEOALIGN_GPS), "r") as fp:
    geoalignment_gps  = json.load(fp)
Rt_gt2geo_gps = (np.array(geoalignment_gps["R_gt2geo"]), np.array(geoalignment_gps["t_gt2geo"]))
Rt_geo2geo_align = (aligner.Rt_slam2geo[0].cpu().numpy(), aligner.Rt_slam2geo[1].cpu().numpy())
# compose the initial GPS alignment and the additional refinement
Rt_gt2geo_refined = (
    Rt_geo2geo_align[0] @ Rt_gt2geo_gps[0],
    Rt_geo2geo_align[1] + Rt_geo2geo_align[0] @ Rt_gt2geo_gps[1],
)

output = {
    "projection": dataset.tile_managers[scene].projection.epsg,
    "R_gt2geo": Rt_gt2geo_refined[0].tolist(),
    "t_gt2geo": Rt_gt2geo_refined[1].tolist(),
}
geoalignment_path = Path(dataset.cfg.local_dir, scene, FILENAME_GEOALIGN_MAPLOC)
write_json(geoalignment_path, output)
!manifold put --overwrite {geoalignment_path} {dataset.cfg.dump_dir.replace("manifold://", "")}/{scene}/{geoalignment_path.name}